In [46]:
#require mdadm, nvme-cli, mkfs.xfs

import subprocess
import os
import platform
import re

mountpoint = '/data'

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def scan_md_array():
    arrays_devs = []
    args = ['cat', '/proc/mdstat']
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out,err = p.communicate()
    array_strs = out.decode('UTF-8').strip().split('\n')
    array_strs = array_strs[1:-1:3]
    for array_str in array_strs:
        print('md array detected : ' + array_str.split(':')[0])
    return array_strs
    
    
def format_drive_in_xfs(device):
    print('formatting ' + device)
    args = ['mkfs.xfs', '-f' , device]
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    return out,err

def umount(device):
    args = ['umount' , device]
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if b'' == err:
        print('Unmounted ' + device)
        return True
    else:
        print('Cannot umount')
        print(args)
        print(err.decode('UTF-8'))
        return False
    return true
        
def format_and_mount(devices):
    index = 0
    devices.sort(key=natural_keys)

    for device in devices:
        umount(device)
        mount_dir = mountpoint + '/disk' + str(index)
        out, err = format_drive_in_xfs(device)

        if b'contains a mounted filesystem' in err:
            print(device + ' is already mounted.')
            continue
        
        elif b'No such file or directory' in err:
            print('Wrong device name ' + device)
            continue
            
        elif b'' != err: 
            print(err)
            continue

        
        if not os.path.exists(mount_dir):
            os.makedirs(mount_dir)

        print('Mounting ' + device + ' to ' + mount_dir)
        args = ['mount',device, mount_dir]
        p = subprocess.Popen(args,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = p.communicate()
        
        if b'' != err:
            print(err)
        
        index += 1
        print('')

def clean_drives(devices):
    for device in devices:
        device = '/dev/' + device.split('[')[0]
        print('Zeroing superblocks')
        args = ['mdadm', '--zero-superblock', device]
        p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out,err = p.communicate()
        if b'' != err:
            print(err)

        print('wipeing filesystem signature for',device )
        args = ['wipefs', '--all', '--force', device]
        p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out,err = p.communicate()

        if b'' != err:
            print(err)
        
        
#require mdadm, wipefs
def stop_array(array):
    array_name = array.split()[0]
    devices = array.split()[4:]
    umount('/dev/' + array_name)
    print('stopping ' + array_name)
    args = ['mdadm','--stop','/dev/'+ array_name]
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out,err = p.communicate()

    if b'mdadm: stopped ' in err:
        print('stopped' , array_name)
        clean_drives(devices)
        return True
    else:
        print('cannot stop md array')
        print(err.decode('UTF-8'))
        return False

#require nvme-cli
def scan_nvme():
    args = ['nvme','list']
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out,err = p.communicate()
    output_str = out.decode('UTF-8')
    
    index = 0 
    devices = []
    for line in output_str.split('\n')[:-1]:
        if index > 1:
            devices.append(line.split()[0])
        index += 1 
    return devices

def install_required_packages():
    distro_name = platform.linux_distribution()[0]
    if 'CentOS' in distro_name or 'Red Hat' in distro_name:
        args = ['yum', 'install', '-y','mdadm', 'nvme-cli', 'xfsprogs','util-linux']
    elif 'Ubuntu' in distro_name or 'Debian' in distro_name:
        args = ['apt','install', '-y','mdadm', 'nvme-cli', 'xfsprogs','util-linux']
    
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out,err = p.communicate()
    print(out.decode('UTF-8'))


In [47]:
if __name__ == '__main__':

    #install_required_packages()
    
    device_blocks = scan_nvme()
    print('NVMe drives found:')
    for device_block in device_blocks:
        print(device_block, end=' ')
    print()
    
    md_arrays = scan_md_array()
    nvme_arrays = []
    
    for md_array in md_arrays:
        for device_block in device_blocks:
            if device_block.split('/')[2] in md_array:
                nvme_arrays.append(md_array)
                break
    
    
    for nvme_array in nvme_arrays:
        print('Do you want to destroy array', nvme_array.split()[0].strip() + '? (y/n)')
        ans = input()
    
        if ans == 'y' or ans == 'Y':
            stop_array(nvme_array)
        else:
            for device in nvme_array.split()[4:]:
                device = '/dev/' + device.split('[')[0]
                device_blocks.remove(device)
    print('')        
    print('formatting', ' '.join(device_blocks))
    format_and_mount(device_blocks)

NVMe drives found:
/dev/nvme0n1 /dev/nvme10n1 /dev/nvme11n1 /dev/nvme12n1 /dev/nvme13n1 /dev/nvme14n1 /dev/nvme15n1 /dev/nvme1n1 /dev/nvme2n1 /dev/nvme3n1 /dev/nvme4n1 /dev/nvme5n1 /dev/nvme6n1 /dev/nvme7n1 /dev/nvme8n1 /dev/nvme9n1 

formatting /dev/nvme0n1 /dev/nvme10n1 /dev/nvme11n1 /dev/nvme12n1 /dev/nvme13n1 /dev/nvme14n1 /dev/nvme15n1 /dev/nvme1n1 /dev/nvme2n1 /dev/nvme3n1 /dev/nvme4n1 /dev/nvme5n1 /dev/nvme6n1 /dev/nvme7n1 /dev/nvme8n1 /dev/nvme9n1
Cannot umount
['umount', '/dev/nvme0n1']
umount: /dev/nvme0n1: not mounted

formatting /dev/nvme0n1
Mounting /dev/nvme0n1 to /data/disk0

Cannot umount
['umount', '/dev/nvme1n1']
umount: /dev/nvme1n1: not mounted

formatting /dev/nvme1n1
Mounting /dev/nvme1n1 to /data/disk1

Unmounted /dev/nvme2n1
formatting /dev/nvme2n1
Mounting /dev/nvme2n1 to /data/disk2

Unmounted /dev/nvme3n1
formatting /dev/nvme3n1
Mounting /dev/nvme3n1 to /data/disk3

Unmounted /dev/nvme4n1
formatting /dev/nvme4n1
Mounting /dev/nvme4n1 to /data/disk4

Unmounted